In [ ]:
%%bash
if [[ ! -d "dataset" ]]; then
  mkdir dataset
  curl -L -o dataset/70-dog-breedsimage-data-set.zip\
    https://www.kaggle.com/api/v1/datasets/download/gpiosenka/70-dog-breedsimage-data-set
  unzip dataset/70-dog-breedsimage-data-set.zip -d dataset
fi

Archive:  dataset/70-dog-breedsimage-data-set.zip
  inflating: dataset/dogs.csv        
  inflating: dataset/test/Afghan/01.jpg  
  inflating: dataset/test/Afghan/02.jpg  
  inflating: dataset/test/Afghan/03.jpg  
  inflating: dataset/test/Afghan/04.jpg  
  inflating: dataset/test/Afghan/05.jpg  
  inflating: dataset/test/Afghan/06.jpg  
  inflating: dataset/test/Afghan/07.jpg  
  inflating: dataset/test/Afghan/08.jpg  
  inflating: dataset/test/Afghan/09.jpg  
  inflating: dataset/test/Afghan/10.jpg  
  inflating: dataset/test/African Wild Dog/01.jpg  
  inflating: dataset/test/African Wild Dog/02.jpg  
  inflating: dataset/test/African Wild Dog/03.jpg  
  inflating: dataset/test/African Wild Dog/04.jpg  
  inflating: dataset/test/African Wild Dog/05.jpg  
  inflating: dataset/test/African Wild Dog/06.jpg  
  inflating: dataset/test/African Wild Dog/07.jpg  
  inflating: dataset/test/African Wild Dog/08.jpg  
  inflating: dataset/test/African Wild Dog/09.jpg  
  inflating: dataset/tes

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  214M  100  214M    0     0   112M      0  0:00:01  0:00:01 --:--:--  158M


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import os
import copy
import time
from torch.optim import lr_scheduler

data_dir = 'dataset'
data_phases = ['train', 'valid']

# Definir transformaciones
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
# Cargar los datasets usando ImageFolder
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in data_phases
}

# Crear DataLoaders
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
    for x in data_phases
}

# Información del dataset (importante para el modelo)
dataset_sizes = {x: len(image_datasets[x]) for x in data_phases}
class_names = image_datasets['train'].classes
num_classes = len(class_names)

# Configuración del dispositivo (CPU/GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Clases detectadas: {num_classes}")
print(f"Imágenes de Entrenamiento: {dataset_sizes['train']}")
print(f"Imágenes de Validación: {dataset_sizes['valid']}")

Clases detectadas: 70
Imágenes de Entrenamiento: 7946
Imágenes de Validación: 700


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Cargar el modelo ResNet18 pre-entrenado
model_ft = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Congelar todas las capas excepto la capa final (opcional pero recomendado inicialmente)
for param in model_ft.parameters():
    param.requires_grad = False

# Reemplazar la capa final (fully-connected)
# ResNet18 tiene un 'fc' (fully-connected) como capa final
num_ftrs = model_ft.fc.in_features
# La nueva capa de clasificación debe tener 'num_classes' (70) salidas
model_ft.fc = nn.Linear(num_ftrs, num_classes)

model_ft = model_ft.to(device)

# 4. Definir función de pérdida y optimizador
# Solo estamos optimizando los parámetros que tienen requires_grad=True
# Si se descongelaron todas las capas (para fine-tuning completo), se optimizan todos.
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 136MB/s]


In [ ]:
# Función principal de entrenamiento
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    """
    Función para entrenar y evaluar el modelo.

    Args:
        model (nn.Module): Modelo a entrenar.
        criterion (nn.Module): Función de pérdida.
        optimizer (optim.Optimizer): Optimizador.
        scheduler (optim.lr_scheduler): Programador de la tasa de aprendizaje.
        num_epochs (int): Número de épocas para entrenar.

    Returns:
        nn.Module: El modelo entrenado con la mejor precisión de validación.
    """
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Iterar sobre las épocas
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 20)

        # Cada época tiene una fase de entrenamiento y una de validación
        for phase in ['train', 'valid']:
            if phase == 'train':
                # Establecer el modelo en modo de entrenamiento
                model.train()
            else:
                # Establecer el modelo en modo de evaluación
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterar sobre los datos. dataloaders[phase] usa 'train' o 'valid'
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Poner a cero los gradientes de los parámetros
                optimizer.zero_grad()

                # Ejecutar el forward pass y backward pass solo si estamos en 'train'
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimizar solo en fase de entrenamiento
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Estadísticas
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Ajustar la tasa de aprendizaje después de la fase de entrenamiento
            if phase == 'train':
                scheduler.step()

            # Calcular el promedio de pérdida y precisión para la época
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy de los pesos del mejor modelo (basado en la precisión de validación)
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'Entrenamiento completado en {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Mejor precisión de validación: {best_acc:.4f}')

    # Cargar los mejores pesos del modelo
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# ReduceLROnPlateau es un scheduler común para Fine-Tuning
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# --- INICIO DEL ENTRENAMIENTO ---
NUM_EPOCHS = 15

model_ft = train_model(
    model_ft,
    criterion,
    optimizer_ft,
    exp_lr_scheduler,
    num_epochs=NUM_EPOCHS
)


Epoch 1/15
--------------------
train Loss: 3.4971 Acc: 0.2352
valid Loss: 2.2148 Acc: 0.6057

Epoch 2/15
--------------------
train Loss: 2.3342 Acc: 0.5318
valid Loss: 1.3502 Acc: 0.8000

Epoch 3/15
--------------------
train Loss: 1.8412 Acc: 0.6248
valid Loss: 0.9901 Acc: 0.8600

Epoch 4/15
--------------------
train Loss: 1.5812 Acc: 0.6728
valid Loss: 0.8343 Acc: 0.8571

Epoch 5/15
--------------------
train Loss: 1.4260 Acc: 0.6947
valid Loss: 0.7310 Acc: 0.8929

Epoch 6/15
--------------------
train Loss: 1.3259 Acc: 0.6954
valid Loss: 0.7033 Acc: 0.8829

Epoch 7/15
--------------------
train Loss: 1.2613 Acc: 0.7056
valid Loss: 0.6554 Acc: 0.9071

Epoch 8/15
--------------------
train Loss: 1.2087 Acc: 0.7197
valid Loss: 0.6558 Acc: 0.8971

Epoch 9/15
--------------------
train Loss: 1.2060 Acc: 0.7219
valid Loss: 0.6542 Acc: 0.9043

Epoch 10/15
--------------------
train Loss: 1.1802 Acc: 0.7260
valid Loss: 0.6512 Acc: 0.9071

Epoch 11/15
--------------------
train Loss: 1.1

In [ ]:
# Guardamos los pesos del modelo entrenado
# torch.save(model_ft.state_dict(), 'resnet18_70_breeds_best_weights.pth')